In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
pd.options.mode.chained_assignment = None

# Exploratory Data Analysis and Preprocessing

In [ ]:
path = "/content/smileannotationsfinal.csv"
df = pd.read_csv(path,names=['id', 'text', 'category'])

In [ ]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [ ]:

df=df[df["category"]!="nocode"]
df

,id,text,category
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
5,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy
...,...,...,...
3078,611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant
3079,612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy
3080,613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy
3082,615246897670922240,@MrStuchbery @britishmuseum Mesmerising.,happy


In [ ]:
df = df[~df.category.str.contains('\|')]

Check for class imbalance after removing nocode and multiple-emotion categories. We can see that our dataset is heavily skewed towards happy


In [ ]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

Create an emotion-label mapping


In [ ]:
possible_labels = df.category.unique()
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
df["label"]=label_encoder.fit_transform(df["category"])
 

In [ ]:
df

,id,text,category,label
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,2
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,2
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy,2
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,2
5,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,2
...,...,...,...,...
3078,611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,3
3079,612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,2
3080,613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,2
3082,615246897670922240,@MrStuchbery @britishmuseum Mesmerising.,happy,2


# Training/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values, 
    df.label.values, 
    test_size = 0.15,
    random_state = 17,
    stratify = df.label.values
)

In [ ]:
df["data_type"] = ["not_set"]*df.shape[0]
df.loc[X_train,"data_type"]='train'
df.loc[X_val,"data_type"]='validation'

In [ ]:
df_group=df.groupby(["category","label","data_type"]).count()
df_group

id  text
category     label data_type            
angry        0     train        48    48
                   validation    9     9
disgust      1     train         5     5
                   validation    1     1
happy        2     train       966   966
                   validation  171   171
not-relevant 3     train       182   182
                   validation   32    32
sad          4     train        27    27
                   validation    5     5
surprise     5     train        30    30
                   validation    5     5

# Loading Tokenizer and Encoding our Data

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 41.1 MB/s 
     |████████████████████████████████| 895 kB 39.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Setting up Bert pre-trained model

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Encoding: Convert text data into numbers.

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
    )
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='validation'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='validation'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=6,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Creating Data Loaders


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

# Setting Up Optimizer and Scheduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
epochs = 30
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,num_warmup_steps=0,num_training_steps=len(dataloader_train)*epochs)

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict   

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

# Defining our Performance Metrics

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')
def accuracy_per_class(preds, labels) :
    label_dict_inverse = {v: k for k, v in label_dict.items()}  
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')   

# Creating our Training Loop

In [ ]:
import random
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.6305905699729919
Validation loss: 0.6161461898258754
F1 Score (Weighted): 0.718100347248411


Epoch 2:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.4962453570216894
Validation loss: 0.6068609314305442
F1 Score (Weighted): 0.7507897804877246


...


Epoch 28
Training loss: 0.0064192187535809355
Validation loss: 0.833881501108408
F1 Score (Weighted): 0.8430297492472342


Epoch 29:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 29
Training loss: 0.005995497838011943
Validation loss: 0.8359209623719964
F1 Score (Weighted): 0.8430297492472342


Epoch 30:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 30
Training loss: 0.005696973920566961
Validation loss: 0.8346596909686923
F1 Score (Weighted): 0.8430297492472342


#  Loading and Evaluating our Model

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 4/9

Class: not-relevant
Accuracy: 0/1

Class: angry
Accuracy: 166/171

Class: disgust
Accuracy: 18/32

Class: sad
Accuracy: 0/5

Class: surprise
Accuracy: 2/5

